In [ ]:
import module_loader
import pandas as pd
from modules.curves import *
from modules.dual import Dual

# Principal Component Analysis Methods

### Build a simple curve for risking and pricing

In [ ]:
nodes = {
    datetime(2022, 1, 1): Dual(1, {"v0": 1}),
    datetime(2024, 1, 1): Dual(1, {"v1": 1}),
    datetime(2027, 1, 1): Dual(1, {"v2": 1}),
    datetime(2032, 1, 1): Dual(1, {"v3": 1}),
    datetime(2052, 1, 1): Dual(1, {"v4": 1}),
}
swaps = {
    Swap(datetime(2022, 1, 1), 12*2, 12, 12): 1.20,
    Swap(datetime(2022, 1, 1), 12*5, 12, 12): 1.66,
    Swap(datetime(2022, 1, 1), 12*10, 12, 12): 1.93,
    Swap(datetime(2022, 1, 1), 12*30, 12, 12): 2.20,
}
s_cv = SolvedCurve(nodes=nodes, interpolation="log_linear", swaps=list(swaps.keys()), obj_rates=list(swaps.values()))
s_cv.iterate()  

### Add a short historical data sample for covariance

In [ ]:
historical_rates = pd.DataFrame({
    "2Y": [1.199, 1.228, 1.210, 1.215, 1.203, 1.159, 1.175, 1.188, 1.159, 1.100],
    "5Y": [1.663, 1.696, 1.665, 1.680, 1.677, 1.657, 1.673, 1.676, 1.653, 1.600],
    "10Y": [1.928, 1.945, 1.934, 1.93, 1.934, 1.931, 1.958, 1.972, 1.932, 1.900],
    "30Y": [2.201, 2.217, 2.228, 2.239, 2.226, 2.235, 2.242, 2.236, 2.22, 2.200],
})
historical_chgs = historical_rates.diff(-1)*100
Q = historical_chgs.cov().to_numpy()
Q

### Create a portfolio of swaps replicating the risks in the text

In [ ]:
portfolio = Portfolio(objects=[
    Swap(datetime(2022,1,1), 12*2, 12, 12, notional=-1.272e6),
    Swap(datetime(2022,1,1), 12*5, 12, 12, notional=313545),
    Swap(datetime(2022,1,1), 12*10, 12, 12, notional=109931),
    Swap(datetime(2022,1,1), 12*30, 12, 12, notional=27160),
])  

### Utilise all the PCA methods displaying results in table

In [ ]:
lambd, E = portfolio.pca(Q)
print(lambd)
print(E)

In [ ]:
portfolio.historical_multipliers(Q, historical_chgs)

In [ ]:
portfolio.risk(s_cv)

In [ ]:
portfolio.pca_risk(s_cv, Q)

In [ ]:
portfolio.pca_covar_alloc(s_cv, Q)

### Hedging trades for directionality

In [ ]:
S_ini = np.array([-10000, 20000, -10000, 0])[:, np.newaxis]
S_ini

The folowing adjustments yield a trade with zero risk to PC1

In [ ]:
portfolio.pca_hedge_adjustment(Q, S_ini)

In [ ]:
print("PCA Risk:\n", np.matmul(E.T, S_ini + portfolio.pca_hedge_adjustment(Q, S_ini)))

The problem with the above minimal adjustment is that it includes a 30Y bucket even though 30Y was not considered in the original trade. We can explicitly set that bucket to be zero in our directionality adjustment.

In [ ]:
portfolio.pca_hedge_adjustment(Q, S_ini, L=[3])

In [ ]:
print("PCA Risk:\n", np.matmul(E.T, S_ini + portfolio.pca_hedge_adjustment(Q, S_ini, L=[3])))

In [ ]:
S_adj = S_ini + portfolio.pca_hedge_adjustment(Q, S_ini, L=[3])
S_adj

If desired we could also negate multiple PCs

In [ ]:
portfolio.pca_hedge_adjustment(Q, S_ini, H=[0,2], L=[3])

In [ ]:
print("PCA Risk:\n", np.matmul(E.T, S_ini + portfolio.pca_hedge_adjustment(Q, S_ini, H=[0,2], L=[3])))

This however, can be quite unstable and can vastly change the initial representation of the trade, which is probably not desired in most cases.

### Example in Text

In [ ]:
market_movement = np.array([17.2, 18.1, 12.5, 8.1])[:, np.newaxis]
print("PnL: ", np.matmul(S_adj.T, market_movement))

In [ ]:
nc_multipliers = np.matmul(centralised_mm.T, E).T
nc_multipliers

In [ ]:
print("PnL from PCA: ", np.matmul(np.matmul(E.T, S_adj).T, nc_multipliers))

In [ ]:
nc_multiplier_neg = nc_multipliers.copy()
nc_multiplier_neg[1,0] = 0
nc_multiplier_neg[2,0] = 0
nc_multiplier_neg[3,0] = 0

In [ ]:
mm_adj = np.matmul(E, multiplier_neg)
mm_adj

In [ ]:
residuals = market_movement - mm_adj
residuals

In [ ]:
(residuals * S_adj).sum()